Learning Outcomes
- Identify and illustrate the legal and ethical issues around the use of supervised machine learning
- Identify formulate and apply the basic processes of supervised machine learning
- Understand the role of data in estimating accuracy 
Lectures: recap then
- 15 minutes: basic model building process: train and test (Validation and model selection are in L2)
- Activity:
- 15 minutes:  types of model: instance-based (kNN) vs explicit (decision trees,rules) 
- Activity:
- 15 minutes: Exaxmple-  greedy rule induction as compared to expert system
- Activity:



# Supervised Machine Learning
### Artificial Intelligence 1, Week 6


### Learning models for **classification** or **regression** from a set of labelled instances.

# This week
Learning outcomes:
- Identify and illustrate the legal and ethical issues around the use of supervised machine learning
- Identify formulate and apply the basic processes of supervised machine learning
- Understand the role of data in estimating accuracy 

Lectures:
- Basic model building process: train and test 
- Types of model: instance-based ( e.g. kNN) vs explicit (e.g. decision trees,rules, ...) 
- Example:   greedy rule induction as compared to expert system



# Machine Learning Paradigm
- Completely different paradigm to symbolic AI
- Create a system with the ability to learn
- Present the system with series of examples
- System builds up its own model of the world



<img src="figures/ML/PersonThinkingAboutDogs.png" syle="float:left" width = 300>

<img src="figures/ML/idealisedDog.png" style="float:right">

## Video (6:52): Hello World of Machine Learning Recipes


https://youtu.be/cKxRvEZd3Mw


## It's all about the data
- Computers cannot experience artefacts of the real world directly
- Instead they just deal with a few variables that represent them
- ML algorithms learn from a “training set” containing digital representations of examples to learn from
- Outcomes depend entirely on:
 - What you choose to measure
 - And how representative your training set is
 



## Example: Top Trumps for Cats and Dogs! 
<img src="figures/ML/TopTrumps.png" width = 80%>

## More formally

We have a set of *n* examples., and for each one  we have: 
- a value for each of *f* features 
- a label

If the label comes from a discrete unordered set of *m* values, e.g.  ("Cat", "Dog"), we have a **Classification** problem.  
We learn a model *M* that is a mapping from a *f*-dimensional continous space (the feature values) onto a finite set   *M*: R<sup>f</sup> --> \{1,...,m\}

If the label is an ordinal value (integer,    floating point) we have a **Regression** problem. *M*:R<sup>f</sup>->R

The data set *X* is a 2-D array of *n* rows and *f* columns. 
The label set *y* is a 2D array with *n* entries

### The next slide shows a function to visualise the data set, the ones after  illustrates this for the Cats and Dogs/IRIS dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def show_scatterplot_matrix(X,y,featureNames,title=None):
    f = X.shape[1]
    if(len(y) != X.shape[0]):
        print("Error,   the y array  must have the same length as there are rows in X")
        return
    fig, ax = plt.subplots(f,f,figsize=(12,12))
    plt.set_cmap('jet')
    for feature1 in range(f):
        ax[feature1,0].set_ylabel( featureNames[feature1])
        ax[0,feature1].set_xlabel( featureNames[feature1])
        ax[0,feature1].xaxis.set_label_position('top') 
        for feature2 in range(f):
            xdata = X[:,feature1]
            ydata = X[:,feature2]
            ax[feature1, feature2].scatter(xdata,ydata,c=y)
    if title != None:
        fig.suptitle(title,fontsize=16,y=0.925)
            

### Cats and Dogs

In [ ]:
columnLabels = ["Top Speed","Max. Length", "Max. Weight","Food Intake","LifeSpan"]

X = np.empty((8,5)) 
X[0] = [48,0.9,11,1,14]   # Red Fox
X[1] = [40,0.6,8,0.75,12] # Arctic Fox
X[2] = [60,2.5,250,20,29] # Jaguar
X[3] = [60,2.5,250,20,29] # Lion
X[4] = [70,1.6,80,10,2]  # "Grey Wolf"
X[5] = [56,1.1,36,4.5,12] # "African Wild Dog"
X[6] = [60,1.9,91,8,21] # Leopard
X[7] = [50,3,320,40,26] # Tiger

title = "All the 2-d slices through the 5-dimensional feature space"
y = [0,0,1,1,0,0,1,1] # y[i] will be 1 for cat,  0 for dog
show_scatterplot_matrix(X,y,columnLabels,title)

In [ ]:
import sklearn.datasets
X,y = sklearn.datasets.load_iris(return_X_y=True)
title="Scatterplots of 2D slices through the 4D Iris data"
show_scatterplot_matrix(X,y,columnLabels,title)

# Timeout?

basic workflow.  train
test

# Recap so far
Machine Learning is about learning patterns from data. In supervised ML this means:

**Training Data**: set of labelled examples, each characterised by values for *f* features  
**X**: data - usually a 2D array with one row per example, one column for each feature  
( even images can be 'flattened' i.e. written out pixel by pixel, row by row into a 1D array for each row).   
**y** : the labels/target 

A supervised Machine Learning **Algorithm**

A **performance criteria**: used ot drive training and then estimate quality of model.  
Dependiong on the **contexzt** this might be accuracy,  precision, recall,...


A **test set** to estimate the performance of the model on unseen data.  
If this is not available separately, have to take out some data from the training set
 - crude way; single 70:30 train:test split, making sure you preserve the proportions of different classes
 - better way: split data into ten repeatedly train on 9/10 test on remaining 1/10, 
 - "headline" result is mean, but keep split results for statistical testing

## Machine Learning Algorithms
Typically a ML method consists of:

1: A  representation for the decision boundaries
 - Each different arrangement of boundaries defines a unique model
 - Each unique model is defined by the set of values for variables specifying where they are
 
2: A learning algorithm to deciding how to change values to move between models
 - last week we saw how the KMeans clustering algorirthm uses "local search with random restarts"

ML Algorithms build models in different ways
- but they don’t care what it is they are grouping
 - and its meaningless to say they “understand”.


## Some example ML methods
The field of ML is fast growing and contains many complex methods and representations
In this module I will just focus on a few simple ideas to give you a feel for what is out there.  
- Instance-based learning (k-Nearest Neighbours) - this week
- Decision trees and rule induction algorithms- this week
- Artificial Neural Networks - weeks 7 and 8

Next year: 
- Artificial Intelligence 2:  15 credits, semester 1 (AI and "General" pathways)
and in particular
- Machine Learning: 15 credits, semester 2     ( AI pathway)

will cover more algorithms in greater depth.


## Instance-based Methods: Nearest Neighbour Methods
- Do not explicitly represent class boundaries  
  Construct them “on-the-fly” when queried
- Store the set of training examples  
  More efficient methods may not store all points
- Use a metric to calculate distance between two points  
  e.g. Euclidean (continuous), Hamming (binary), ...


## K-Nearest Neighbour Classification <img src="figures/ML/voronoi.png" style="float:right" width = 400>

**init()**  :Specify a distance metric d(i,j) for any two items *i* and *j*     
  e.g. Euclidean (continuous variables) or Hamming (categorical)

**fit(trainingData)** :
Just store a local copy of the training data as two arrays:  
  X_train of size (N * numFeatures),  
  y_train of size( N * 1)
  
**Predict(newItems)**

    *Step 1*:   Make 2D array distances of size (num_newItems * N)  
    For each newItem i  
        for each trainingitem j  
            set distances [i] [j] = d (i,j) 

    *Step 2*: Get labels of the k nearest neighbours  
    For each newItem i  
        Find the *k* columns for row i with the smallest values  
            Get the corresponding *k* labels from y_train  

    *Step 3*: Store majority vote in a  1D array y_pred of size (numToPredict *1)   
        For each newItem i  
            For each label m  
                count votes amongst the k Nearest neightbour of i  
            Set y_pred[i] = value of m with highest count
 
    return y_pred

Image adapted from Vornoi tesselation for kNN from https://kevinzakka.github.io/2016/07/13/k-nearest-neighbor


In [ ]:
# Example for K = 1 
from sklearn.metrics.pairwise import euclidean_distances
class simple_1NN:

    def fit(self,X,y):
        self.numExemplars = X.shape[0]
        self.numFeatures = X.shape[1]
        self.modelX = X
        self.modelY = y
        
    def predict(self,newItems):
        numToPredict = newItems.shape[0]
        yPred = np.empty((numToPredict,1))
        dist = euclidean_distances(newItems,self.modelX)
        closest = np.argmin(dist, axis=1) 
        for item in range (numToPredict):
            yPred[item] = self.modelY [ closest[item]]
        return yPred


## Cats and dogs

In [ ]:
# Scottish wildcat 14kg, 0.8m, 48kph, 8 years, 1.5kg food a day

model = simple_1NN()
model.fit(X,y)

newAnimals = np.empty((2,5))
scottish_wildcat = np.array([14,0.8,48,8,1.5])
newAnimals[0] = scottish_wildcat

prediction = model.predict(newAnimals)

if(prediction[0]==0):
    decision="dog"
else:
    decision ="cat"
    
print(" the scottish wildcat is a " + decision)



##  What are the problems here?



### Cats and Dogs

1. Some variables (e.g. weight) have bigger ranges that otherrs so dominate the distance calculation
 - solve this by **normalising** the data so that each feature is mapped onto the range [0-1]
 
2.   Our training data is not very representative
 - it doesn't even include the house cat!
 
Problem 1 can be addressed by algorithmic methods  - and is standard practice

Problem 2 can only be addressed by good practice.  
It relates to the issues of:
- **Fairness**:  big cats may be exciting but they are not representative of cats as whole
- **Accountability**: can you explain the decision.  
  Actually "for these features a wildcat is most like a red fox" is not too bad
- **Trust**: can you make a convincing argument that you have carefully chosen the right examples **and features**

## K-NN works just as well as more complex algorithms  for many data sets

### Illustrating k-NN on a classic data set: Iris flowers <img src="figures/ML/Iris-image.png" style="float:right">
- classic Machine Learning Data set
- 4 measurements: sepal and petal width and length
- 50 examples  from each 3 sub-specieis for iris flowers
- three class problem:
 - so for some trypes of alghortityhm have to decide whether to a 3-way classifier or bnest 1-vs-rest classifers
- most ML classifieers can get over 90%




## Iris

We'll use a function from sklearn to do our train/test split here.

This is handy because it shiuffles the data and has options to make sure that we keep the same proportion of different classes in our training anad testing data.

The alternative woul be to do it by hand e.g.  since we know the data is written out by classes first

    X_train=np.empty((0,4))
    y_train=[]
    X_test=np.empty((0,4))
    y_test=[]
    for i in range(X.shape[0]):
        if ( i%3==0):
            X_test=np.vstack((X_test,X[i]))
            y_test.append(y[i])
        else:
            X_train= np.vstack((X_train,X[i]))
            y_train.append(y[i])
            
           
We'll also make a **confusiuon matrix** to examine the predictions it makes
rows = target labels,  columns = predicted labels
           

In [ ]:
# make train/test split 
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np

X,y = load_iris(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y)


model = simple_1NN()
model.fit(X_train,y_train)
ypred = model.predict(X_test)
confusionMatrix = np.zeros((3,3),int)
for i in range(50):
    actual = int(y_test[i])
    predicted = int(ypred[i])
    confusionMatrix[actual][predicted] += 1
print(confusionMatrix)


# Timeout

## Rule Induction Algorithms

In Topic One we looked at 'Knowledge-Based systems' where **humans provided the rules** for a situation.

In supervised learning we are interested in how we can make **machines learn the rules** for an application.  
To do that we need to be able to:

1 Have a way of assigning "goodness" to (sets of) rules.

2 Algorithmically generate possible rules
  - Often, we have fixed sets of features,operators, and outputs, and can discretize the thresholds for each feature.  
    So we can use nested loops to create all the possible rules as tuples of {feature_id,operator_id,threshold_id, label_id}
  - Typically exploit this in a greedy constructive hill climbing approach:  
    Repeatedly generate all the rules we could add to existing set of rules (model),   
    Then select and adding the one that discriminates most of the remaining unclassified data 

- Most existing algorithms tend to use rules built up of lots of axis-perpendicular decisions.  e.g.,*If( maxWeight > 100) THEN ("cat")*   
  Draws a line through feature space, perpendicular to the maxWeight axis, crossing it at 100.  
  Puts the label "Cat" on one side, "Dog" on the other

- As more rules are added, the model effectively builds labelled (hyper) boxes in space.  
  Rest of space is given with the default (majority) label






## Example for cats and dogs in 2D <img src="figures/ML/cat-dogs-rules.png" style="float:right" width=500>
Chart shows scatter plot of lifespan (y) vs top speed 

Start by adding rule that classifies most examples: 
- **if(lifespan < 15) type = dog**

Then add next best rule to discriminate rest 
- **if(speed >65) type = dog**

This example the model learnt consits of the following  three rules:

`IF (lifespan < 15 ) THEN type = "dog" 
ELSE IF (speed > 65) THEN type = "dog" 
ELSE (default) type = "cat" `


# Pseudocode
rule = {variable, operator, threshold, prediction}  
type candidateSolution = list of rules + score   

Score() uses ruleset in candidate solution to make predictions on training set  
 and returns -1 if any errors,  else number of correct predictions
 
**Note that a set of rules may not cover every training example**

    Preprocess (trainingset)  
    SET workingCandidate=[], score = 0   
    WHILE (workingCandidate.score<trainingsetSize) DO  
        SET tmp = workingCandidate; //make a copy so we can repeatedly edit it  
        SET nextWC = workingCandidate //store the most promising offspring  
        FOR newRule in  (all_possible_rules)  
            SET workingCandidate = tmp //reset to original  
            SET workingCandidate = workingCandiudate + newRule //extend working candidate
            Score(workingCandidate)  
            IF  workingCandidate.score > nextWC.score  
                SET nextwc = workingCandidate    
        SET workingCandidate = nextWC  
    RETURN workingCandidate  


## Timeout

## Decision Trees <img src="figures/ML/cats-dogs-tree.png" style= "float:right">
Tree-based structure can capture rules and more.

Basic idea: divide input space using a set of axis-parallel lines by **"growing"** a tree

1. Start with single node that predicts majority class label.
2. Recursively:
 1. measure the "data purity"  or "information content"  
  of the data that arrives at that node
 2. examine each way of splitting data  you could put into that node
 3. measure the information content of ther l;eft and right chil nodes you would get
 4. if the  "best" split is above some threshold then add it and repeat

**Interior nodes** are equivalent to conditions in a rule  
**Leaf Nodes** are the outputs: class labels (classification tree) or equation for predicting values (regression tree)


## Decision trees example on the iris data
using code from sklearn 
`class sklearn.tree.DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort='deprecated', ccp_alpha=0.0)`

Like all sklearn models it implements a fit() and predict() method

Note the default criteria for splitting is the 'gini' indes = there are many available, this is a popular one


In [ ]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# load iris dataset and split into train:test
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y)


model = DecisionTreeClassifier(random_state=1234)
model.fit(X_train,y_train)
ypred = model.predict(X_test)
confusionMatrix = np.zeros((3,3),int)
for i in range(50):
    actual = int(y_test[i])
    predicted = int(ypred[i])
    confusionMatrix[actual][predicted] += 1
print(confusionMatrix)



fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(model, 
                   feature_names=iris.feature_names,  
                   class_names=iris.target_names,
                   filled=True)

## So how do  we learn models?
**Construction**:  add boundaries to make models more complex
- Add examples to kNN
- Repeatedly add nodes to trees, splitting on new variables
- Repeatedly add rules that classify as-yet unclassified data
 - Add nodes to an artifical neural network
 
**Perturbation**: Move existing boundaries to change model
- Change value of K or distance function in kNN
- Change rule/treenode thresholds: *if lifespan < 15*  => *if lifespan < 18*
- Change operators in rules/ tree nodes:  *if lifespan < 15* => *if lifespave &leq; 15*
- Change variables considered in rules/tree nodes: *if lifespan < 15* => *if dailyfood < 15*
- Change weights in MLP, 


## Summary
Supervised Machine Learning is concerned with learning predictive models from datasets
- Different algorithms use different representations of decision boundaries
- inside the boundaries are:
 - **Class labels** for a classification problem
 - **(formulas leading to) continuous values** for a regression problem

Algorithms **fit** models to data by repeatedly:
  - making and testing small changes,  
  - and then selecting the ones that improve accuracy on the training set
  - until some stop criteria is met

  - They do this by either adding complexity or changing the parameters of an existing model
  - This is equivalent to moving through “model space”

Once the model has been learned (fit) we leave it unchanged  
  - and use it to **predict** the labels for new data points

Next week:   Neural Networks


## Fruit

In [ ]:
# importthe apples-ornages-bananas data set from the first week
# class labels are 0:apple, 1:orange 2: banana
# columns in X are Red,Green,Blue,Width,Height,Weight,Type
import  numpy as np
from sklearn.model_selection import train_test_split
# read in all thedata fro mthe apples-oranges-banas dataset
alldata = np.genfromtxt('data/fruits2.csv', delimiter=',')

#pull out the first two  feature vales and the labels into two differnt arrays

X = alldata[:,:-1]
y = alldata[:,-1]


numItems = X.shape[0]

train_x,test_x,train_y,test_y = train_test_split(X,y,test_size=0.33,stratify=y)


model = simple_1NN()
model.fit(train_x,train_y)


ypred = model.predict(test_x)
confusionMatrix = np.zeros((3,3),int)
for i in range(len(test_y)):
    actual = int(test_y[i])
    predicted = int(ypred[i])
    confusionMatrix[actual][predicted] += 1
print(confusionMatrix)


